In [1]:
#import le module rsa
import rsa

def generateKeys():
    (publicKey, privateKey) = rsa.newkeys(1024)
    with open("publicKey1.pem", 'wb') as p:
        p.write(publicKey.save_pkcs1("PEM"))
    with open("privateKey1.pem", 'wb') as p:
        p.write(privateKey.save_pkcs1("PEM"))


In [2]:
# reload keys
def loadKeys():
    with open("publicKey1.pem", 'rb') as p:
        publicKey = rsa.PublicKey.load_pkcs1(p.read())
    with open("privateKey1.pem", 'rb') as p:
        privateKey = rsa.PrivateKey.load_pkcs1(p.read())
    return publicKey, privateKey

In [3]:
#decrypt
def encrypt(message, key):
    return rsa.encrypt(message.encode('ascii'), key)

#epcryption
def decrypt(cyphertext, key):
    try:
        return rsa.decrypt(cyphertext, key).decode("ascii")
    except:
        return False

In [4]:
#main programme
generateKeys()

publicKey1, privateKey1 = loadKeys()

In [5]:
print(publicKey1, '\n\n')
print(privateKey1)

PublicKey(96185371164923691440231772115434227591769531388228239706061727048559491547761419020730382427085090577041206700715175453540901977105418385999554141517392024948439904799124006884298290874242943378875347298239688409176914821404385170265363715741440970008344333509097120955196779742322231203492155264227756403313, 65537) 


PrivateKey(96185371164923691440231772115434227591769531388228239706061727048559491547761419020730382427085090577041206700715175453540901977105418385999554141517392024948439904799124006884298290874242943378875347298239688409176914821404385170265363715741440970008344333509097120955196779742322231203492155264227756403313, 65537, 38095788323052996913409159845588388934488330656640377466930806246844663657250761455683026941420078674767819740459035495179229940640025406188724323543560270969818972873913837460683333754526393316825215287880536107498762094910702634377845985918048178727666340340127711626960849189296205537147734732815833044993, 33104466437301599896298294776232

In [6]:
#message = input("input your message:")
message = 'test'
cypherText = encrypt(message, publicKey1)

In [7]:
text = decrypt(cypherText, privateKey1)

if text:
    print(f"message text : {text}")
else:
    print(f"unable to decrypt the message")

message text : test


In [8]:
#signature
def sign(message, key):
    return rsa.sign(message.encode('ascii'), key, 'SHA-1')

#verifier
def verify(message, signature, key):
    try:
        return rsa.verify(message.encode('ascii'), signature)
    except:
        return False

In [9]:
signature = sign(message, privateKey1)
print(f'signature : {signature}')

signature : b'K\xc7s\xb6\xfb\xf7=\xf1\xd0y\xc6\x81\x17\x95F\x95\xb8\xa5\xe2\x0c\xad\xa3\xc1f\x846)\x1a\x9d|+ \x16eK6*\xf1%\n\xb2]A\xd34$\xc5hNt\xf1\xaa\xce\x02\x8f\x9b\xeb\x95\x94-0\xea\x91D\x98\xf7[\x91\x15.+\xc2\xba\xca\xb5Y\x87\\\x95^\xe35S6\xce\xbc\xb6\x93\xae\x8d\xd0\xfc\x0f.\xcbZq0vW\x1e\x05\xb2\xd0?:\x9fgT\xf4\xd7*\xd0\xa5\x83\x06\xc6\xab.\xc94\x15Y\x82\x96~\x94\xe1'


In [10]:
if verify (text, signature, publicKey1):
    print("succefully verified signature")
else:
    print("the message signature could not be verified")


the message signature could not be verified


In [ ]:
#echo-server.py
import json
import socket
import time

HOST = "127.0.0.1"
PORT = 65432

# Dictionnaire pour stocker les informations des clients
clients_info = {}


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()

    while True:
        # Si "exit" est reçu, le serveur sort de la boucle
        print("____________________________________________________________________________")
        print("Choose the client you are receiving a message from (either 1 or 2.. more clients to be added later).\nNOTE : You can type exit if you want to shut down the server.")
        x = input()
        if x == "exit":
            break
        elif (x != '1') and (x != '2'):
            print("\nI gave you only 3 options, why would you type something else?\n\n")
            time.sleep(1)

        else:

            conn, addr = s.accept()
            with conn:
                #print(f'Connected by {addr}')
                with open("privateKey1.pem", 'rb') as p:
                    privateKey = rsa.PrivateKey.load_pkcs1(p.read())
                # Étape une: Échange de messages cryptés
                data = conn.recv(1024)

                data = decrypt(data, privateKey)
                print(f"\n\nReceived : {data}\n\n")

                # Étape deux: Enregistrement des informations du client
                clients_info[addr] = data
                with open("clients_info.txt", 'a') as info_file:
                    info_file.write(f"Client: {data}, Address: {addr}\n")
                    client_id = data

                # Envoi des informations du client au client
                message = f'You are {client_id}, I received your message!'
                if x == '1':
                     with open("publicKey2.pem", 'rb') as p:
                            publicKey = rsa.PublicKey.load_pkcs1(p.read())
                elif x == '2':
                     with open("publicKey3.pem", 'rb') as p:
                            publicKey = rsa.PublicKey.load_pkcs1(p.read())
               
                response = encrypt(message, publicKey)
                print(f"Sent : {response}")
                conn.sendall(response)

                # Attente de la réponse de l'accusé de réception
                response = conn.recv(1024)
                decrypted_response = decrypt(response, privateKey)
                print(f"\n\nReceived acknowledgment : {decrypted_response}\n\n")
                print("____________________________________________________________________________")



____________________________________________________________________________
Choose the client you are receiving a message from (either 1 or 2.. more clients to be added later).
NOTE : You can type exit if you want to shut down the server.
1


Received : Client 1,192.168.1.1


Sent : b'\x0e\xde{\\\xea\x8c\x19Qa\xbd\xfaG\x86J)h\xf6T\x8b`\x98\x10Y\x7fM?\xc5\xf3\xbc+\xe3\xf1\xb2@\x88kk\xe8g\xf2"\x05\x82\x8c!\xa3B\xd4\x8c\xce\xf3V(a\xe6\xbc\xfb\x83Uv{\xe4\x9f\x87\xc8 OI\x95\xfc%b\x08)R\'\xc5\xa0M3]M\xe1*\x0c\x92\xe5\x92Y\xcd\xb8W}7\xed\xce\x9b(\xb1\xe2oX\x1aH;vPo\xf3\xa6\xb1\xee\xf0\xd8\xb7\t\xae\xfa\xad4\xc7\xb6@B\x06\xf8l['


Received acknowledgment : This is sent from Client 1,192.168.1.1 - Acknowledgment received!


____________________________________________________________________________
____________________________________________________________________________
Choose the client you are receiving a message from (either 1 or 2.. more clients to be added later).
NOTE : You can type